In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import sklearn.linear_model as lm
import seaborn as sns
from tqdm.notebook import tqdm

In [2]:
np.random.seed(42)
sample_size = 100
mpg = sns.load_dataset('mpg')
print("Full Data Size:", len(mpg))
mpg_sample = mpg.sample(sample_size)
print("Sample Size:", len(mpg_sample))
px.scatter(mpg_sample, x='weight', y='mpg', trendline='ols', width=800)

Full Data Size: 398
Sample Size: 100


In [3]:
model = lm.LinearRegression().fit(mpg_sample[['weight']], mpg_sample['mpg'])
model.coef_

array([-0.00730597])

In [4]:
def estimator(sample):
  model = lm.LinearRegression().fit(sample[['weight']], sample['mpg'])
  return model.coef_[0]

In [9]:
def bootstrap(sample, statistic, num_repetitions):
  stats = []
  for i in tqdm(np.arange(num_repetitions), "Bootstrapping"):
    bootstrap_sample = sample.sample(frac=1, replace=True)
    bootstrap_stat = statistic(bootstrap_sample)
    stats.append(bootstrap_stat)
  return stats

In [10]:
bs_thetas = bootstrap(mpg_sample, estimator, 10000)

Bootstrapping:   0%|          | 0/10000 [00:00<?, ?it/s]

In [11]:
fig = px.histogram(pd.Series(bs_thetas, name="Bootstrap Distribution"),
                   title='Bootstrap Distribution of the Slope',
                   width=800, histnorm='probability',
                   barmode="overlay", opacity=0.8)
fig.add_vline(0)

In [12]:
def bootstrap_ci(bootstrap_samples, confidence_level=95):

  lower_percentile = (100 - confidence_level) / 2
  upper_percentile = 100 - lower_percentile

  return np.percentile(bootstrap_samples, [lower_percentile, upper_percentile])

In [13]:
bootstrap_ci(bs_thetas)

array([-0.00814752, -0.00653232])

In [14]:
ci_line_style = dict(color="orange", width=2, dash="dash")
fig.add_vline(x=bootstrap_ci(bs_thetas)[0], line=ci_line_style)
fig.add_vline(x=bootstrap_ci(bs_thetas)[1], line=ci_line_style)

In [15]:
mpg_pop = sns.load_dataset('mpg')
theta_est = [estimator(mpg_pop.sample(sample_size)) for i in tqdm(range(10000))]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [16]:
print("Actual CI", bootstrap_ci(theta_est))
fig.add_histogram(x=theta_est, name='Population Distribution', histnorm='probability', opacity=0.7)
fig.add_vline(x=bootstrap_ci(theta_est)[0], line=dict(color="red", width=2, dash="dash"))
fig.add_vline(x=bootstrap_ci(theta_est)[1], line=dict(color="red", width=2, dash="dash"))

Actual CI [-0.00852048 -0.00691081]


In [17]:
thetas = pd.DataFrame({"bs_thetas": bs_thetas, "thetas":theta_est})
px.histogram(thetas.melt(), x='value', facet_row='variable',
             title='Distribution of the Slope', width=800)

In [ ]:
csv_file = 'data/Full24hrdataset.csv.gz'
usecols = ['Date', 'ID',  'region', 'PM25FM', 'PM25cf1', 'TempC', 'RH', 'Dewpoint']
full_df = pd.read_csv(csv_file, usecols=usecols, parse_dates=['Date']).dropna()
full_df.columns = ['date', 'id', 'region', 'pm25aqs', 'pm25pa', 'temp', 'rh', 'dew']
full_df = full_df[(full_df['pm25aqs'] < 50)]
bad_dates = pd.to_datetime(['2019-08-21', '2019-08-22', '2019-09-24'])
GA = full_df[(full_df['id'] == 'GA1') & (~full_df['date'].isin(bad_dates))]
GA = GA.sort_values("pm25aqs")
display(full_df["region"].value_counts())
display(GA.head())
print("Number of Rows:", GA.shape[0])

In [ ]:
model = lm.LinearRegression().fit(GA[['pm25aqs']], GA['pm25pa'])
theta_0, theta_1 = model.intercept_, model.coef_[0]

In [ ]:
fig = px.scatter(GA, x='pm25aqs', y='pm25pa', width=800)
xtest = pd.DataFrame({"pm25aqs": np.array([GA['pm25aqs'].min(), GA['pm25aqs'].max()])})
fig.add_scatter(x=xtest["pm25aqs"], y=model.predict(xtest[["pm25aqs"]]), mode='lines',
                name="Least Squares Fit")

In [ ]:
print(f"True Air Quality Estimate = {-theta_0/theta_1:.2} + {1/theta_1:.2}PA")

In [ ]:
model2 = lm.LinearRegression().fit(GA[['pm25pa']], GA['pm25aqs'])

fig = px.scatter(GA, y='pm25aqs', x='pm25pa', width=800)
xtest["pm25pa"] = np.array([GA['pm25pa'].min(), GA['pm25pa'].max()])
fig.add_scatter(x=xtest["pm25pa"], y=xtest["pm25pa"] *1/theta_1 - theta_0/theta_1, mode='lines',
                name="Inverse Fit")
fig.add_scatter(x=xtest["pm25pa"], y=model2.predict(xtest[['pm25pa']]), mode='lines',
                name="Least Squares Fit")

In [ ]:
model_h = lm.LinearRegression().fit(GA[['pm25aqs', 'rh']], GA['pm25pa'])
[theta_1, theta_2], theta_0 = model_h.coef_, model_h.intercept_

print(f"True Air Quality Estimate = {-theta_0/theta_1:1.2} + {1/theta_1:.2}PA + {-theta_2/theta_1:.2}RH")

In [ ]:
fig = px.scatter(GA, x='pm25aqs', y='pm25pa', width=800)
fig.add_scatter(x=xtest['pm25aqs'], y=model.predict(xtest[['pm25aqs']]), mode='lines',
                name="Least Squares Fit")
fig.add_scatter(x=GA["pm25aqs"], y=model_h.predict(GA[['pm25aqs']]), mode='lines+markers',
                marker_size=5, name="Least Squares")

In [ ]:
fig = px.scatter_3d(GA, x='pm25aqs', y='rh', z='pm25pa', width=800, height=600)

grid_resolution = 2
(u,v) = np.meshgrid(
    np.linspace(GA["pm25aqs"].min(), GA["pm25aqs"].max(), grid_resolution),
    np.linspace(GA["rh"].min(), GA["rh"].max(), grid_resolution))
zs = model_h.predict(pd.DataFrame({"pm25aqs": u.flatten(), "rh": v.flatten()}))
zs_old = model.predict(pd.DataFrame({"pm25aqs": u.flatten()}))
color1 = px.colors.qualitative.Plotly[3]
color2 = px.colors.qualitative.Plotly[4]
fig.add_surface(x=u, y=v, z=zs.reshape(u.shape), opacity=1,
                colorscale=[[0, color1], [1, color1]],
                showscale=False, name="AQS + RH")
fig.add_surface(x=u, y=v, z = zs_old.reshape(u.shape), opacity=1,
                colorscale=[[0, color2], [1, color2]],
                showscale=False, name="AQS")
fig.update_scenes(aspectmode='cube')

In [ ]:
theta_1

In [ ]:
theta_2

In [2]:
def theta2_estimate(sample):
  model = lm.LinearRegression().fit(sample[['pm25aqs', 'rh']], sample['pm25pa'])
  return model.coef_[1]

In [ ]:
bs_theta2 = bootstrap(GA, theta2_estimate, 10000)

In [ ]:
import plotly.express as px
fig = px.histogram(x=bs_theta2,
                   labels=dict(x='Bootstrapped Humidity Coefficient'),
                   histnorm='probability',
                   width=800)
fig.add_vline(0)
fig.add_vline(x=bootstrap_ci(bs_theta2)[0], line=ci_line_style)
fig.add_vline(x=bootstrap_ci(bs_theta2)[1], line=ci_line_style)

In [ ]:
len([elem for elem in bs_theta2 if elem < 0.0])

In [ ]:
eggs = pd.read_csv('data/snowy_plover.csv.gz')
eggs.head()

In [ ]:
eggs.shape

In [ ]:
y = eggs["bird_weight"]
X = eggs[["egg_weight", "egg_length", "egg_breadth"]]

model = lm.LinearRegression(fit_intercept=True).fit(X, y)

display(pd.DataFrame(
    [model.intercept_] + list(model.coef_),
    columns=['theta_hat'],
    index=['intercept', 'egg_weight', 'egg_length', 'egg_breadth']))

all_features_rmse = np.mean((y - model.predict(X)) ** 2)

print("RMSE", all_features_rsme)

In [ ]:
def all_thetas(sample):
  model = lm.LinearRegression().fit(
      sample[["egg_weight", "egg_length", "egg_breadth"]],
      sample["bird_weight"])
  return [model.intercept_] + model.coef_.tolist()

In [ ]:
bs_thetas = pd.DataFrame(
    bootstrap(eggs, all_thetas, 10_000),
    columns=['intercept', 'egg_weight', 'egg_length', 'egg_breadth'])
bs_thetas

In [ ]:
cis = (bs_thetas
       .apply(bootstrap_ci).T
       .rename(columns={0: 'lower', 1: 'upper'}))
cis

In [ ]:
from operator import sub
def visualize_coeffs(bs_thetas, rows, cols):
  cis = (bs_thetas
         .apply(bootsrap_ci).T
         .rename(columns={0: 'lower', 1: 'upper'}))
  display(cis)
  from plotly.subplots import make_subplots
  fig = make_subplots(rows=rows, cols=cols, subplot_titles=cis.index)
  for i, coeff_name in enumerate(cis.index):
    c = (i % cols) + 1
    r = (i // cols) + 1
    fig.add_histogram(x=bs_thetas[coeff_name], name=coeff_name,
                      row=r, col=c, histnorm='probability')
    fig.add_vline(x=0, row=r, col=c)
    fig.add_vline(x=cis.loc[coeff_name, 'lower'], line=ci_line_style,
                  row=r, col=c)
    fig.add_vline(x=cis.loc[coeff_name, 'upper'], line=ci_line_style,
                  row=r, col=c)
  return fig

In [ ]:
visualize_coeffs(bs_thetas, 2,2)

In [ ]:
px.scatter_matrix(eggs, width=600, height=600)

In [ ]:
px.imshow(eggs.corr().round(2), text_auto=True, width=600)

In [ ]:
U, s, Vt = np.linalg.svd(eggs[['egg_weight', 'egg_length', 'egg_breadth']])
px.line(s)

In [ ]:
px.scatter(eggs, x='egg_weight', y='bird_weight', trendline='ols', width=800)

In [ ]:
y = eggs["bird_weight"]
X = eggs[["egg_weight"]]

model = lm.LinearRegression(fit_intercept=True).fit(X, y)

display(pd.DataFrame([model.intercept_] + list(model.coef_),
                     columns=['theta_hat'],
                     index=['intercept', 'egg_weight']))
print("All Features RMSE", all_features_rmse)
print("RMSE", np.mean((y - model.predict(X)) ** 2))

In [ ]:
def egg_weight_coeff(sample):
  model = lm.LinearRegression().fit(
      sample[["egg_weight"]],
      sample["bird_weight"])
  return [model.intercept_] + model.coef_.tolist()

In [ ]:
bs_thetas_egg_weight = pd.DataFrame(
    bootstrap(eggs, egg_weight_coeff, 10_000),
    columns = ['intercept', 'egg_weight'])
bs_thetas_egg_weight

In [ ]:
visualize_coeffs(bs_thetas_egg_weight, 1, 2)